In [1]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import os

# --- Configuración de la Página ---
st.set_page_config(page_title="Análisis de Entrenamiento YOLOv8", layout="wide")
st.title("📊 Visor de Resultados de Entrenamiento YOLOv8")

# --- 1. Cargar los Datos ---
CSV_PATH = 'results.csv'

@st.cache_data
def load_data(path):
    """Carga el archivo results.csv en un DataFrame de pandas."""
    if not os.path.exists(path):
        st.error(f"Error: No se encontró el archivo '{path}'.")
        st.error("Por favor, asegúrate de que 'results.csv' esté en la misma carpeta que 'app.py'.")
        return None
    try:
        df = pd.read_csv(path)
        # Limpiar espacios en blanco de los nombres de las columnas si existen
        df.columns = df.columns.str.strip()
        return df
    except Exception as e:
        st.error(f"Error al leer el archivo CSV: {e}")
        return None

df = load_data(CSV_PATH)

if df is not in None:
    st.success("Archivo 'results.csv' cargado exitosamente.")
    
    # --- 2. Mostrar Métricas Clave (KPIs) ---
    st.subheader("Resultados Principales")
    
    try:
        # Encontrar el mejor mAP (el más estricto)
        best_map_95_epoch = df['metrics/mAP50-95(B)'].idxmax()
        best_map_95_score = df['metrics/mAP50-95(B)'].max()
        
        # Encontrar el mejor mAP50 (el más común)
        best_map_50_epoch = df['metrics/mAP50(B)'].idxmax()
        best_map_50_score = df['metrics/mAP50(B)'].max()
        
        total_epochs = df['epoch'].max()
        
        col1, col2, col3 = st.columns(3)
        col1.metric("Épocas Totales", f"{total_epochs}")
        col2.metric("Mejor mAP50-95 (Época)", f"{best_map_95_score:.4f} (en Época {best_map_95_epoch + 1})")
        col3.metric("Mejor mAP50 (Época)", f"{best_map_50_score:.4f} (en Época {best_map_50_epoch + 1})")
        
    except KeyError as e:
        st.error(f"Error: La columna {e} no se encontró. El CSV podría estar mal formado.")
    except Exception as e:
        st.error(f"Ocurrió un error al calcular las métricas: {e}")

    # --- 3. Gráficos de Pérdida (Loss) ---
    st.subheader("Curvas de Pérdida (Loss)")
    st.markdown("""
    **Cómo leer este gráfico:** La "Pérdida" (Loss) mide cuán equivocado está el modelo.
    * **Curvas Azules (train):** Qué tan bien el modelo memoriza los datos de entrenamiento.
    * **Curvas Naranjas (val):** Qué tan bien el modelo *generaliza* a datos nuevos (esto es lo más importante).
    * **Objetivo:** Quieres que ambas líneas (train y val) bajen juntas. Si 'val' (naranja) empieza a subir mientras 'train' (azul) sigue bajando, es un síntoma de **sobreajuste (overfitting)**.
    """)
    
    
    
    try:
        fig_loss, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 6))
        
        # Gráfico de Pérdida de Caja (Box Loss)
        ax1.plot(df['epoch'], df['train/box_loss'], label='train_box_loss', color='blue')
        ax1.plot(df['epoch'], df['val/box_loss'], label='val_box_loss', color='orange', linestyle='--')
        ax1.set_title('Pérdida de Caja (Box Loss) vs. Épocas')
        ax1.set_xlabel('Época')
        ax1.set_ylabel('Pérdida')
        ax1.legend()
        ax1.grid(True)
        
        # Gráfico de Pérdida de Clase (Class Loss)
        ax2.plot(df['epoch'], df['train/cls_loss'], label='train_cls_loss', color='blue')
        ax2.plot(df['epoch'], df['val/cls_loss'], label='val_cls_loss', color='orange', linestyle='--')
        ax2.set_title('Pérdida de Clase (Class Loss) vs. Épocas')
        ax2.set_xlabel('Época')
        ax2.set_ylabel('Pérdida')
        ax2.legend()
        ax2.grid(True)
        
        st.pyplot(fig_loss)
        
    except KeyError:
        st.warning("No se pudieron generar los gráficos de Pérdida (Loss). Faltan columnas en el CSV.")

    # --- 4. Gráficos de Métricas (mAP, Precisión, Recall) ---
    st.subheader("Curvas de Métricas de Validación")
    st.markdown("""
    **Cómo leer estos gráficos:**
    * **mAP (Mean Average Precision):** La métrica principal. mAP50-95 es muy estricta, mAP50 es más estándar. ¡Más alto es mejor!
    * **Precision (Precisión):** De todas las detecciones que hizo el modelo, ¿cuántas fueron correctas?
    * **Recall (Sensibilidad):** De todos los graffitis *reales* que había en las imágenes, ¿cuántos encontró el modelo?
    """)
    
    
    
    try:
        fig_metrics, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 6))
        
        # Gráfico de mAP
        ax1.plot(df['epoch'], df['metrics/mAP50(B)'], label='mAP50(B)', color='green')
        ax1.plot(df['epoch'], df['metrics/mAP50-95(B)'], label='mAP50-95(B)', color='red', linestyle='--')
        ax1.set_title('mAP vs. Épocas')
        ax1.set_xlabel('Época')
        ax1.set_ylabel('Puntuación mAP')
        ax1.legend()
        ax1.grid(True)
        
        # Gráfico de Precisión y Recall
        ax2.plot(df['epoch'], df['metrics/precision(B)'], label='Precision(B)', color='purple')
        ax2.plot(df['epoch'], df['metrics/recall(B)'], label='Recall(B)', color='cyan', linestyle='--')
        ax2.set_title('Precisión y Recall vs. Épocas')
        ax2.set_xlabel('Época')
        ax2.set_ylabel('Puntuación')
        ax2.legend()
        ax2.grid(True)
        
        st.pyplot(fig_metrics)
        
    except KeyError:
        st.warning("No se pudieron generar los gráficos de Métricas. Faltan columnas en el CSV.")
        
    # --- 5. Mostrar Datos Crudos ---
    st.subheader("Datos Crudos")
    with st.expander("Ver la tabla 'results.csv' completa"):
        st.dataframe(df)

SyntaxError: invalid syntax (2544972273.py, line 31)